# Renset Classification on CIFAR 10

In this example, we show you how to use torchvision model to make a federated classification task

## Dataset: CIFAR 10

You can download the CIFAR-10 dataset through this link: 
- https://webank-ai-1251170195.cos.ap-guangzhou.myqcloud.com/fate/examples/data/cifar-10.zip
  
The origin CIFAR-10 is from: 
- https://www.cs.toronto.edu/~kriz/cifar.html
  
For the convinence of demonstrate, our clients will use same dataset

In [7]:
pwd

'/data/projects/fate/persistence/fate'

In [2]:
import sys
sys.path.append('/data/projects/fate/persistence/fate/python/fate_client')
sys.path.append('/data/projects/fate/persistence/fate/python')

import os
os.chdir("/data/projects/fate/persistence/fate")
os.environ['FATE_PROJECT_BASE'] = '/data/projects/fate/persistence/fate'

## Local Test

Firstly we locally test our model and dataset. If it works, we can submit a federated task.

In [3]:
from pipeline.component.nn import save_to_fate

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%%save_to_fate model resnet.py

# model
import torch as t
from torch import nn
from torchvision.models import resnet18, ResNet18_Weights

class Resnet(nn.Module):

    def __init__(self, ):
        super(Resnet, self).__init__()
        self.resnet = resnet18()
        self.classifier = t.nn.Linear(1000, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        if self.training:
            return self.classifier(self.resnet(x))
        else:
            return self.softmax(self.classifier(self.resnet(x)))
    

In [5]:
model = Resnet()
print(model)

Resnet(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

In [8]:
# read dataset
from federatedml.nn.dataset.image import ImageDataset

ds = ImageDataset()
ds.load('/data/projects/fate/persistence/fate/examples/data/cifar10/train/')

In [9]:
ds[0][0].shape

torch.Size([3, 32, 32])

In [13]:
# local test
import torch as t
from federatedml.nn.homo.trainer.fedavg_trainer import FedAVGTrainer

trainer = FedAVGTrainer(epochs=1, batch_size=512, data_loader_worker=2)
trainer.set_model(model)

optimizer = t.optim.Adam(model.parameters(), lr=0.001)
loss = t.nn.CrossEntropyLoss()

trainer.local_mode() # set local mode
trainer.train(ds, None, optimizer, loss)

ipcl_python failed to import
epoch is 0
100%|██████████| 98/98 [00:35<00:00,  2.76it/s]
epoch loss is 1.4889685620117188


## Submit a federated task

In [34]:
import torch as t
from torch import nn
from pipeline import fate_torch_hook
from pipeline.component import HomoNN
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, Evaluation, DataTransform
from pipeline.interface import Data, Model

fate_torch_hook(t)

import os
fate_project_path = os.path.abspath('/data/projects/fate')
guest = 9999
host = 10000
pipeline = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest, host=host,
                                                                            arbiter=host)
data_0 = {"name": "cifar10", "namespace": "experiment"}
data_path = fate_project_path + '/examples/data/cifar10/train'
pipeline.bind_table(name=data_0['name'], namespace=data_0['namespace'], path=data_path)
pipeline.bind_table(name=data_0['name'], namespace=data_0['namespace'], path=data_path)

{'namespace': 'experiment', 'table_name': 'cifar10'}

In [35]:
reader_0 = Reader(name="reader_0")
reader_0.get_party_instance(role='guest', party_id=guest).component_param(table=data_0)
reader_0.get_party_instance(role='host', party_id=host).component_param(table=data_0)

reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role='guest', party_id=guest).component_param(table=data_0)
reader_1.get_party_instance(role='host', party_id=host).component_param(table=data_0)

In [36]:
from pipeline.component.homo_nn import DatasetParam, TrainerParam

model = t.nn.Sequential(
    t.nn.CustModel(module_name='resnet', class_name='Resnet')
)

nn_component = HomoNN(name='nn_0',
                      model=model, 
                      loss=t.nn.CrossEntropyLoss(),
                      optimizer = t.optim.Adam(lr=0.001, weight_decay=0.001),
                      dataset=DatasetParam(dataset_name='image'),  # 使用自定义的dataset
                      trainer=TrainerParam(trainer_name='fedavg_trainer', epochs=1, batch_size=512, data_loader_worker=2),
                      torch_seed=100
                      )

In [37]:
pipeline.add_component(reader_0)
pipeline.add_component(reader_1)
pipeline.add_component(nn_component, data=Data(train_data=reader_0.output.data, validate_data=reader_1.output.data))
pipeline.add_component(Evaluation(name='eval_0', eval_type='multi'), data=Data(data=nn_component.output.data))

In [38]:
pipeline.compile()

In [ ]:
pipeline.fit() # submit pipeline here

2023-02-20 07:45:59.834 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202302200745594166160

2023-02-20 07:45:59.842 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
m2023-02-20 07:46:00.864 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-02-20 07:46:00.865 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:01
2023-02-20 07:46:01.885 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2023-02-20 07:46:02.902 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-02-20 07:46:03.919 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-02-20 07:46:04.939 | INFO     

2023-02-20 07:46:37.772 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:37
2023-02-20 07:46:38.814 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:38
2023-02-20 07:46:39.866 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:40
2023-02-20 07:46:40.890 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:41
2023-02-20 07:46:41.914 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:42
2023-02-20 07:46:42.945 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:43
2023-02-20 07:46:43.987 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 

2023-02-20 07:47:19.104 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:01:19
2023-02-20 07:47:20.125 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:01:20
2023-02-20 07:47:21.145 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:01:21
2023-02-20 07:47:22.172 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:01:22
2023-02-20 07:47:23.192 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:01:23
2023-02-20 07:47:24.233 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:01:24
2023-02-20 07:47:25.264 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 

2023-02-20 07:47:59.916 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:02:00
2023-02-20 07:48:00.935 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:02:01
2023-02-20 07:48:01.951 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:02:02
2023-02-20 07:48:02.968 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:02:03
2023-02-20 07:48:03.987 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:02:04
2023-02-20 07:48:05.006 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:02:05
2023-02-20 07:48:06.030 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 

2023-02-20 07:48:40.620 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:02:40
2023-02-20 07:48:41.635 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:02:41
2023-02-20 07:48:42.652 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:02:42
2023-02-20 07:48:43.668 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:02:43
2023-02-20 07:48:44.685 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:02:44
2023-02-20 07:48:45.702 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:02:45
2023-02-20 07:48:46.717 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 

...

2023-02-20 08:44:16.600 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:58:16
2023-02-20 08:44:17.617 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:58:17
2023-02-20 08:44:18.633 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:58:18
2023-02-20 08:44:19.649 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:58:19
2023-02-20 08:44:20.665 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:58:20
2023-02-20 08:44:21.684 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:58:21
2023-02-20 08:44:22.700 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 

2023-02-20 08:44:57.289 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:58:57
2023-02-20 08:44:58.304 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:58:58
2023-02-20 08:44:59.319 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:58:59
2023-02-20 08:45:00.335 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:59:00
2023-02-20 08:45:01.352 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:59:01
2023-02-20 08:45:02.367 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:59:02
2023-02-20 08:45:03.384 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 

2023-02-20 08:45:37.946 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:59:38
2023-02-20 08:45:38.966 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:59:39
2023-02-20 08:45:39.983 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:59:40
2023-02-20 08:45:40.999 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:59:41
2023-02-20 08:45:42.015 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:59:42
2023-02-20 08:45:43.032 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:59:43
2023-02-20 08:45:44.048 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 

2023-02-20 08:46:18.606 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 1:00:18
2023-02-20 08:46:19.622 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 1:00:19
2023-02-20 08:46:20.640 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 1:00:20
2023-02-20 08:46:21.656 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 1:00:21
2023-02-20 08:46:22.671 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 1:00:22
2023-02-20 08:46:23.692 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 1:00:23
2023-02-20 08:46:24.708 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 

2023-02-20 08:46:59.282 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 1:00:59
2023-02-20 08:47:00.303 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 1:01:00
2023-02-20 08:47:01.320 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 1:01:01
2023-02-20 08:47:02.337 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 1:01:02
2023-02-20 08:47:03.352 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 1:01:03
2023-02-20 08:47:04.368 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 1:01:04
2023-02-20 08:47:05.383 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 